# Import libraries

In [59]:
# http://datalytics.ru/all/kak-ispolzovat-google-bigquery-s-pomoschyu-python/
import pandas as pd
import os
import pandas_profiling
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] ='data/dev-rep-final-projects-326152cab3fb.json'

In [ ]:
# !pip install --upgrade google-cloud-bigquery[pandas]

In [5]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [6]:
from google.cloud import bigquery
client = bigquery.Client()

In [52]:
# Test connections
sql = """
SELECT *
FROM `dev-rep-final-projects.au_weather.location_labels` AS location_labels
LIMIT 15
"""

df = client.query(sql).to_dataframe()
print(df.shape)
df.head()

(15, 2)


,Location,Location_ID
0,Adelaide,0
1,Albany,1
2,Albury,2
3,AliceSprings,3
4,BadgerysCreek,4


# main query

In [56]:

sql = """
SELECT  cloud_info.Date,
        cloud_info.Location_ID,
        location_labels.Location,
        cloud_info.Cloud9am,
        cloud_info.Cloud3pm,
        humidity_info.Humidity9am,
        humidity_info.Humidity3pm,
        pressure_info.Pressure9am,
        pressure_info.Pressure3pm,
        rain_info.RISK_MM,
        rain_info.RainTomorrow,
        temperature_info.MinTemp,
        temperature_info.MaxTemp,
        temperature_info.Temp9am,
        temperature_info.Temp3pm,
        weather_info.Rainfall,
        weather_info.Evaporation,
        weather_info.Sunshine,
        weather_info.RainToday,
        wind_info.WindGustDir,
        wind_info.WindGustSpeed,
        wind_info.WindDir9am,
        wind_info.WindDir3pm,
        wind_info.WindSpeed9am,
        wind_info.WindSpeed3pm,
        
humidity_info.Date AS Data2, humidity_info.Location_ID AS Location_ID2
FROM `dev-rep-final-projects.au_weather.cloud_info` AS cloud_info
LEFT JOIN `dev-rep-final-projects.au_weather.humidity_info` AS humidity_info
USING(Date, Location_ID)
LEFT JOIN `dev-rep-final-projects.au_weather.pressure_info` AS pressure_info
USING(Date, Location_ID)
LEFT JOIN `dev-rep-final-projects.au_weather.rain_info` AS rain_info
USING(Date, Location_ID)
LEFT JOIN `dev-rep-final-projects.au_weather.temperature_info` AS temperature_info
USING(Date, Location_ID)
LEFT JOIN `dev-rep-final-projects.au_weather.weather_info` AS weather_info
USING(Date, Location_ID)
LEFT JOIN `dev-rep-final-projects.au_weather.wind_info` AS wind_info
USING(Date, Location_ID)
LEFT JOIN `dev-rep-final-projects.au_weather.location_labels` AS location_labels
USING(Location_ID)



"""
# ON cloud_info.Date = humidity_info.Date
df = client.query(sql).to_dataframe()
print(df.shape)
df.head()

(142193, 27)


,Date,Location_ID,Location,Cloud9am,Cloud3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,RISK_MM,...,Sunshine,RainToday,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Data2,Location_ID2
0,2008-12-02,2,Albury,None,None,44.0,25.0,1010.6,1007.8,0.0,...,None,False,WNW,44.0,NNW,WSW,4.0,22.0,2008-12-02,2
1,2008-12-04,2,Albury,None,None,45.0,16.0,1017.6,1012.8,1.0,...,None,False,NE,24.0,SE,E,11.0,9.0,2008-12-04,2
2,2008-12-06,2,Albury,None,None,55.0,23.0,1009.2,1005.4,0.0,...,None,False,WNW,56.0,W,W,19.0,24.0,2008-12-06,2
3,2008-12-08,2,Albury,None,None,48.0,19.0,1013.4,1010.1,0.0,...,None,False,W,35.0,SSE,W,6.0,17.0,2008-12-08,2
4,2008-12-09,2,Albury,None,None,42.0,9.0,1008.9,1003.6,1.4,...,None,False,NNW,80.0,SE,NW,7.0,28.0,2008-12-09,2


In [58]:
df.to_csv('data/weather.csv', sep=',')

In [25]:
# %%bigquery
# SELECT cloud.Date, humidity.Date AS Data2
# FROM `dev-rep-final-projects.au_weather.cloud_info` AS cloud
# LEFT JOIN `dev-rep-final-projects.au_weather.humidity_info` AS humidity
# ON cloud.Date = humidity.Date
# LIMIT 5;

,Date,Data2
0,2008-12-02,2008-12-02
1,2008-12-02,2008-12-02
2,2008-12-02,2008-12-02
3,2008-12-02,2008-12-02
4,2008-12-02,2008-12-02


# pandas_profiling

In [60]:
profile = df.profile_report(title='Pandas Profiling Weather Report')
profile.to_file(output_file="data/weather.html")